Programa que, dados los conjuntos de S/N definidos anteriormente:
1) A cada observación de A1, de 112 MHz de BW, la separa en dos sub-observaciones, cada una de 56 MHz de BW. Luego calcula el RMS para cada una de esas dos subandas.
2) Al error en el TOA de cada observación de A2 lo multiplica por un factor sqrt(2), y calcula el RMS correspondiente a los nuevos errores.
3) Grafica RMS en función de S/N para las observaciones originales y para las osbervaciones modificadas.

In [ ]:
import numpy as np

import decimal
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import subprocess
import math

import os
import shutil

In [2]:
#----------------------------------------------------------------------------------------
# Función que da los argumentos para pat en caso de que se desee subdividir en frecuencia
#----------------------------------------------------------------------------------------
def pat_args_subf(nsubint):

    usingtemplate = './template_A1.std'
    
    return '-A PGS -f "tempo2" -s {} -jTD -j "F {}" '.format(usingtemplate,nsubint)

In [3]:
# Leemos el template usado para cada antena

template_A1 = np.loadtxt( './templates/obs_template_A1.dat' , skiprows=0 , dtype=str , usecols=(0) )
template_A2 = np.loadtxt( './templates/obs_template_A2.dat' , skiprows=0 , dtype=str , usecols=(0) )

# Creamos carpetas para guardar los resultados

os.chdir("./results_sn_A1/")
if not os.path.exists('./timing_bw'):
    os.mkdir('./timing_bw')
os.chdir('..') 

# copiamos también el archivo .par a las nueva carpeta,ya que después lo necesitaremos para usar tempo2

shutil.copy('./J0437-4715.par', "./results_sn_A1/timing_bw/")
    
# Estos son arreglos que usaremos después para graficar
sn_array = np.zeros(6)
rms_A1 = np.zeros(6)
rms_A1_1sub = np.zeros(6)
rms_A1_2sub = np.zeros(6)
rms_A2_2pol = np.zeros(6)
rms_A2_1pol = np.zeros(6)

n_free_A1 = np.zeros(6)
n_free_A2 = np.zeros(6)

In [4]:
#------------------------------------------------------------------------------------------------
# Primero generamos un .tim para cada conjunto de observaciones tomadas con A1,
# donde cada observación de 112 MHz de BW es separada en dos sub-observaciones de 56 MHz de BW
#------------------------------------------------------------------------------------------------

i=0
for y in(1,50,80,110,140,170):
    
    sn_array[i] = y
    
    if y == 1:
        sn = 'total'
    else:
        sn = str(y)

# Cargamos las observaciones correspondientes al conjunto de sn en cuestión
    
    timfile_A1 = './results_sn_A1/timing_files/timing_' + str(sn) + '.tim'
    pfds_A1 = np.loadtxt( timfile_A1 , skiprows=1 , dtype=str , usecols=(0) )
    
    n_free_A1[i] = len(pfds_A1)
    
# Definimos el archivo de salida

    subints_f_toa= './subints_f_' + str(sn) + '.tim' 

    pat_output_subints_f= '>> {}'.format(subints_f_toa)  # nombre del archivo de salida

# Escogemos un número de subteintervalos de frecuencias en los que separar a las observaciones

    nsubints_f = 2
    
# Creamos el archivo .tim
    
    os.chdir('./A1/')   # entramos en la carpeta que contiene los .pfd
    for pfd in pfds_A1:            # recorremos todas las observaciones
    
        subprocess.check_call(['pat '+pat_args_subf(nsubints_f)+pfd+pat_output_subints_f], shell=True)
    
    os.chdir('..')                 # salimos de la carpeta que contiene los .pfd
    
    timing_file = subints_f_toa.replace("./", "")                                           # arreglamos el nombre del archivo de salida
    shutil.copy('./A1/' + timing_file, "./results_sn_A1/timing_bw/")             # copiamos el archivo .tim a la carpeta
    
# si las observaciones son de A1, hay que hacer una corrección por la frecuencia

    subprocess.check_call(["sed -i -e 's/1414.8750/1414.4375/g' ./results_sn_A1/timing_bw/" + timing_file], shell=True)
    
    i += 1

In [4]:
# Las observaciones de cada sub-banda aparecen mezcladas en los archivos .tim generados,
# así que vamos a separarlas.

for y in [1, 50, 80, 110, 140, 170]:

    if y == 1:
        sn = 'total'
    else:
        sn = str(y)
        
# abrimos el archivo tim generado y lo leemos

    timing_folder = "./results_sn_A1/timing_bw/"    # nombre del archivo .tim original
    f = open(timing_folder + "subints_f_" + sn + ".tim")
    lines=f.readlines()
    
# generamos dos archivos de salida, uno por cada sub-banda

    f1 = open(timing_folder + "subints_f1_" + str(sn) + ".tim", "w+")
    f2 = open(timing_folder + "subints_f2_" + str(sn) + ".tim", "w+")
    
    print("FORMAT 1" , file = f1)
    print("FORMAT 1" , file = f2)
    
# escribimos las observaciones de cada sub-banda en sus respectivos archivos de salida
i=1
while i < len(lines):
    f1.write(str(lines[i]))
    i += 1
    
    f2.write(str(lines[i]))
    i += 2
    
f.close
f1.close
f2.close

<function TextIOWrapper.close>

In [ ]:
# Calculamos los residuos para cada una de las sub-bandas

i=0
for y in [1, 50, 80, 110, 140, 170]:
    
    if y == 1:
        sn = 'total'
    else:
        sn = str(y)
        
    resfile = "results_A1_" + sn + ".dat"
    timfile = "timing_" + sn + ".tim"
    subprocess.check_call( "tempo2 >" + resfile + " -us -f J0437-4715.par " + timfile, cwd='./results_sn_A1/timing_files/', shell = True)
    rms_A1[i] = np.genfromtxt ( './results_sn_A1/timing_files/' + resfile, comments="none", dtype=float, skip_header=18, max_rows=1, usecols=(10) )
        
    resfile = "results_subint_f1_" + sn + ".dat"
    timfile = "subints_f1_" + sn + ".tim"
    subprocess.check_call( "tempo2 >" + resfile + " -us -f J0437-4715.par " + timfile, cwd=timing_folder, shell = True)
    rms_A1_1sub[i] = np.genfromtxt ( timing_folder + resfile, comments="none", dtype=float, skip_header=18, max_rows=1, usecols=(10) )
    
    resfile = "results_subint_f2_" + sn + ".dat"
    timfile = "subints_f2_" + sn + ".tim"
    subprocess.check_call( "tempo2 >" + resfile + " -us -f J0437-4715.par " + timfile, cwd=timing_folder, shell = True)
    rms_A1_2sub[i] = np.genfromtxt ( timing_folder + resfile, comments="none", dtype=float, skip_header=18, max_rows=1, usecols=(10) )
    
    i += 1

In [ ]:
#------------------------------------------------------------------------------------------------
# Ahora generamos un .tim para cada conjunto de observaciones tomadas con A2,
# en error en el TOA de cada observación es multiplicado por sqrt(2)
#------------------------------------------------------------------------------------------------
i=0
for y in [1, 50, 80, 110, 140, 170]:
    
    if y == 1:
        sn = 'total'
    else:
        sn = str(y)
    
# abrimos el archivo .tim en el que vamos a crear    

    timing_folder = "./results_sn_A2/timinb_bw/"
    old_timfile = './results_sn_A2/timing_files/timing_' + str(sn) + '.tim'    # nombre del archivo .tim original
    new_timfile =  timing_folder + "sqrt_" + str(sn) + ".tim"                       # nombre de .tim que vamos a crear
    
    f_new = open(timing_folder + new_timfile, "w+")
    factor = math.sqrt(2)                                                      # factor por el que vamos a multiplicar los errores
    
# leemos el archivo .tim original

    pfd_file = np.loadtxt( old_timfile , skiprows=1 , dtype=str , usecols=(0) )
    freq = np.loadtxt( old_timfile, skiprows=1, dtype=float , usecols=(1) )
    MJD = np.loadtxt( old_timfile , skiprows=1 , dtype=decimal.Decimal , usecols=(2) )
    error = np.loadtxt( old_timfile , skiprows=1 , dtype=float,  usecols=(3) )
    telescope = np.loadtxt( old_timfile, skiprows=1, dtype=str, usecols=(4) )
    
    n_free_A2[i] = len(pfd_file)

# escribimos los resultados en el archivo .tim nuevo, pero con la columna de errores multiplicad apor el factor
    print("FORMAT 1", file = f_new)
    for j in range(len(error)):
        
        newerror = error[j] * factor
  
        f_new.write( str(pfd_file[j]) + ' ' + '{:11.6f}'.format(freq[j]) + ' ' + str(MJD[j]) + '   ' + '{:5.3f}'.format(newerror) + '  ' + str(telescope[j]) + '\n')
        
    f_new.close
    f_old.close
    
    i += 1

In [ ]:
#------------------------------------------------------------
# calculamos el rms de las nuevas observaciones usando tempo2
#------------------------------------------------------------

i=0
for y in [1, 50, 80, 110, 140, 170]:
    
    if y == 1:
        sn = 'total'
    else:
        sn = str(y)
        
    resfile = "results_A2_" + sn + ".dat"
    timfile = "timing_" + sn + ".tim"
    subprocess.check_call( "tempo2 >" + resfile + " -us -f J0437-4715.par " + timfile, cwd='./results_sn_A2/timing_files/', shell = True)
    rms_A2_2pol[i] = np.genfromtxt ( './results_sn_A2/timing_files/' + resfile, comments="none", dtype=float, skip_header=18, max_rows=1, usecols=(10) )

    resfile = "results_sqrt_" + sn + ".dat"
    timfile = "sqrt_" + str(sn) + ".tim"
    subprocess.check_call( "tempo2 >" + resfile + " -us -f J0437-4715.par " + timfile, cwd=timing_folder, shell = True)
    rms_A1_1pol[i] = np.genfromtxt ( timing_folder + resfile, comments="none", dtype=float, skip_header=18, max_rows=1, usecols=(10) )

In [ ]:
# graficamos los resultados

# Set the fonts. Always use big fonts.
nice_fonts = {
        # Use LaTeX to write all text
        "text.usetex": True,
        "font.family": "serif",
        # Use 10pt font in plots, to match 10pt font in document
        "axes.labelsize": 30,
        "font.size": 28,
        "axes.linewidth": 1.5,
        # Make the legend/label fonts a little smaller
        "legend.fontsize": 28.5,
        "xtick.labelsize": 26,
        "ytick.labelsize": 26,
}
# Update the fonts
mpl.rcParams.update(nice_fonts)
#plt.tight_layout()

plt.close('all')

plt.rc('text', usetex=True)
plt.rc('font', family='serif', size="12")
plt.rc('lines', linewidth=1.2)

fig, ax = plt.subplots(1, 1, gridspec_kw={'hspace': 0.0, 'wspace': 0.0},figsize=(16,22))

plt.setp(ax, xticks=[1,50,80,110,140,170])

ax.set(xlabel='S/N_{\mathrm{min}}', ylabel="$ {\mathrm{RMS_{new}}}~[\mu s] $")
ax.grid()

# calculamos las barras de error

rms_max_A1 = np.zeros(6)
rms_min_A1 = np.zeros(6)
rms_max_A1_1sub = np.zeros(6)
rms_min_A1_1sub = np.zeros(6)
rms_max_A1_2sub = np.zeros(6)
rms_min_A1_2sub = np.zeros(6)
rms_max_A2_2pol = np.zeros(6)
rms_min_A2_2pol = np.zeros(6)
rms_max_A2_1pol = np.zeros(6)
rms_min_A2_1pol = np.zeros(6)


# calculamos las barras de error de los RMS
# es IMPORTANTE notar que los valores obtenidos no son las cantidades "delta" que debe sumarse/restarse al valor central RMS,
# sino que se obtienen directamente los valores extremos de las barras, es decir, "RMS +/- delta"


for x in range(0,6):
    
    prob_84_A1 = scipy.stats.chi2.ppf( 0.84, n_free_A1[x])
    prob_16_A1 = scipy.stats.chi2.ppf( 0.16, n_free_A1[x])
    
    prob_84_A2 = scipy.stats.chi2.ppf( 0.84, n_free_A2[x])
    prob_16_A2 = scipy.stats.chi2.ppf( 0.16, n_free_A2[x])

    rms_max_A1[x] = round(math.sqrt( ((n_free_A1[x]) / prob_16_A1] )) * rms_A1[x] , 4)
    rms_min_A1[x] = round(math.sqrt( ((n_free_A1[x]) / prob_84_A1 )) * rms_A1[x] , 4)
    
    rms_max_A1_1sub[x] = round(math.sqrt( ((n_free_A1[x]) / prob_16_A1 )) * rms_A1_1sub[x] , 4)
    rms_min_A1_1sub[x] = round(math.sqrt( ((n_free_A1[x]) / prob_84_A1 )) * rms_A1_1sub[x] , 4)
    
    rms_max_A1_2sub[x] = round(math.sqrt( ((n_free_A1[x]) / prob_16_A1 )) * rms_A1_2sub[x] , 4)
    rms_min_A1_2sub[x] = round(math.sqrt( ((n_free_A1[x]) / prob_84_A1 )) * rms_A1_2sub[x] , 4)
    
    rms_max_A2_2pol[x] = round(math.sqrt( ((n_free_A2[x]) / prob_16_A2 )) * rms_A2_2pol[x] , 4)
    rms_min_A2_2pol[x] = round(math.sqrt( ((n_free_A2[x]) / prob_84_A2 )) * rms_A2_2pol[x] , 4) 

    rms_max_A2_1pol[x] = round(math.sqrt( ((n_free_A2[x]) / prob_16_A2 )) * rms_A2_1pol[x] , 4)
    rms_min_A2_1pol[x] = round(math.sqrt( ((n_free_A2[x]) / prob_84_A2 )) * rms_A2_1pol[x] , 4)
    
asymmetric_error_A1 = [abs(rms_min_A1-rms_A1), abs(rms_max_A1-rms_A1)]
asymmetric_error_A1_1sub = [abs(rms_min_A1_1sub-rms_A1_1sub), abs(rms_max_A1_1sub-rms_A1_1sub)]
asymmetric_error_A1_2sub = [abs(rms_min_A1_2sub-rms_A1_2sub), abs(rms_max_A1_2sub-rms_A1_2sub)]
asymmetric_error_A2_2pol = [abs(rms_min_A2_2pol-rms_A2_2pol), abs(rms_max_A2_2pol-rms_A2_2pol)]
asymmetric_error_A2_1pol = [abs(rms_min_A2_1pol-rms_A2_1pol), abs(rms_max_A2_1pol-rms_A2_1pol)]
    
# graficamos

transp = 0.5
dx = 1.2

ax.scatter(sn, rms_A1, label='A1 (112MHz)', alpha=transp, marker="s", lw=8, color="blue")
ax.scatter(sn+dx, rms_A1_1sub, label='A1 sub-banda inf. (56MHz)',  alpha=transp, marker="v", lw=8, color="green")
ax.scatter(sn-dx, rms_A1_2sub, label='A1 sub-banda sup. (56MHz)',  alpha=transp, marker="D", lw=8, color="purple")
ax.scatter(sn, rms_A2_2pol, label='A2 2 polarizaciones',  alpha=transp, marker="^", lw=8, color="darkorange")
ax.scatter(sn, rms_A2_1pol, label='A2 1 polarizacion',  alpha=transp, marker="*", lw=8, color="red")

ax.plot(sn, rms_A1, alpha=transp, marker="s", lw=4, color="blue")
ax.plot(sn+dx, rms_A1_1sub, alpha=transp, marker="v", lw=4, color="green")
ax.plot(sn-dx, rms_A1_2sub, alpha=transp, marker="D", lw=4, color="purple")
ax.plot(sn, rms_A2_2pol, alpha=transp, marker="^", lw=4, color="darkorange")
ax.plot(sn, rms_A2_1pol, alpha=transp, marker="*", lw=4, color="red")

ax.errorbar(sn, rms_A1, yerr=asymmetric_error_A1, alpha=transp, marker="s", fmt='o', capthick=3, capsize=5, color='blue', lw=5)
ax.errorbar(sn+dx, rms_A1_1sub, yerr=asymmetric_error_A1_1sub, alpha=transp, marker="s", fmt='o', capthick=3, capsize=5, color='green', lw=5)
ax.errorbar(sn-dx, rms_A1_2sub, yerr=asymmetric_error_A1_2sub, alpha=transp, marker="s", fmt='o', capthick=3, capsize=5, color='purple', lw=5)
ax.errorbar(sn, rms_A2_2pol, yerr=asymmetric_error_A2_1pol, alpha=transp, marker="s", fmt='o', capthick=3, capsize=5, color='darkorange', lw=5)
ax.errorbar(sn, rms_A2_1pol, yerr=asymmetric_error_A2_2pol, alpha=transp, marker="s", fmt='o', capthick=3, capsize=5, color='red', lw=5)


# remove the errorbars

# use them in the legend
#ax.legend(bbox_to_anchor=(1.445, 1.02))
ax.legend(loc='upper right')


plt.savefig('rms_bw.pdf', bbox_inches='tight')
plt.show()